## Python DBUtils、PyMysql 创建连接池
　　Python 编程中可以使用 PyMysql 进行数据库的连接及诸如查询/插入/更新等操作，但是每次连接 MySQL 数据库请求时，都是独立的去请求访问，相当浪费资源，而且访问数量达到一定数量时，对 mysql 的性能会产生较大的影响。因此，实际使用中，通常会使用数据库的连接池技术，来访问数据库达到资源复用的目的。<br>
　　DBUtils 是一套用于管理数据库连接池的Python包，为高频度高并发的数据库访问提供更好的性能，可以自动管理连接对象的创建和释放。并允许对非线程安全的数据库接口进行线程安全包装。<br>
DBUtils提供两种外部接口：<br>
>PersistentDB ：提供线程专用的数据库连接，并自动管理连接。<br>
PooledDB ：提供线程间可共享的数据库连接，并自动管理连接。<br>

　　实测证明 PersistentDB 的速度是最高的，但是在某些特殊情况下，数据库的连接过程可能异常缓慢，而此时的PooledDB则可以提供相对来说平均连接时间比较短的管理方式。<br>
　　另外，实际使用的数据库驱动也有所依赖，比如SQLite数据库只能使用PersistentDB作连接池。<br>
　　**安装:**
  >pip3 install DBUtils<br>
  
　　在程序创建连接的时候，可以从一个空闲的连接中获取，不需要重新初始化连接，提升获取连接的速度；关闭连接的时候，把连接放回连接池，而不是真正的关闭，所以可以减少频繁地打开和关闭连接。<br>
　　**说明：**<br>
BDUtils数据库链接池：<br>
>模式一：基于threaing.local实现为每一个线程创建一个连接，关闭是伪关闭，当前线程可以重复<br>
模式二：连接池原理<br>
　　　　如果有三个线程来数据库中获取连接：<br>
　　　　　　如果三个同时来的，一人给一个连接；<br>
　　　　　　如果一个一个来，有时间间隔，用一个连接就可以为三个线程提供服务；<br>
　　　　　　其他情况：<br>
　　　　　　　　有可能：1个连接就可以为三个线程提供服务<br>
　　　　　　　　有可能：2个连接就可以为三个线程提供服务<br>
　　　　　　　　有可能：3个连接就可以为三个线程提供服务<br>
>一些参数注释：<br>
mincached ：启动时开启的空连接数量<br>
maxcached ：连接池最大可用连接数量<br>
maxshared ：连接池最大可共享连接数量<br>
maxconnections ：最大允许连接数量<br>
blocking ：达到最大数量时是否阻塞<br>
maxusage ：单个连接最大复用次数<br>
setsession ：用于传递到数据库的准备会话，如 [”set name UTF-8″] <br>
    
    PS：maxshared在使用pymysql中均无用。链接数据库的模块：只有threadsafety>1的时候才有用。<br>
**模式一：**为每个线程创建一个连接，线程即使调用了close方法，也不会关闭，只是把连接重新放到连接池，供自己线程再次使用。当线程终止时，连接自动关闭。（如果线程比较多还是会创建很多连接，推荐使用模式二）<br>
>import pymysql<br>
from DBUtils.PersistentDB import PersistentDB<br>

>POOL = PersistentDB(<br>
　　　　　　creator=pymysql,  # 使用链接数据库的模块<br>
　　　　　　maxusage=None,    # 一个链接最多被重复使用的次数，None表示无限制<br>
　　　　　　setsession=[],    # 开始会话前执行的命令列表。如：["set datestyle to ...", "set time zone ..."]<br>
　　　　　　ping=0,<br>
　　　　　　\# ping MySQL服务端，检查是否服务可用。如：0 = None = never, 1 = default = whenever it is requested, 2 = when a cursor is created, 4 = when a query is executed, 7 = always<br>
　　　　　　closeable=False,<br>
　　　　　　\# 如果为False时，conn.close() 实际上被忽略，供下次使用，再线程关闭时，才会自动关闭链接。如果为True时， conn.close()则关闭链接，那么再次调用pool.connection时就会报错，因为已经真的关闭了连接（pool.steady_connection()可以获取一个新的链接）<br>
　　　　　　threadlocal=None, # 本线程独享值得对象，用于保存链接对象，如果链接对象被重置<br>
　　　　　　host="127.0.0.1",<br>
　　　　　　port=3306,<br>
　　　　　　user="root",<br>
　　　　　　password="",<br>
　　　　　　database="code_record",<br>
　　　　　　charset="utf8"<br>
　　　　)<br>

>def func():<br>
　　　conn = POOL.connection(shareable=False)<br>
　　　cursor = conn.cursor()<br>
　　　cursor.execute("select * from userinfo")<br>
　　　ret = cursor.fetchall()<br>
　　　print(ret)<br>
　　　cursor.close()<br>
　　　conn.close()<br>
>func()<br>

**模式二：**创建一批连接到连接池，供所有线程共享使用，使用完毕后再放回到连接池。（由于pymysql、MySQLdb等threadsafety值为1，所以该模式连接池中的线程会被所有线程共享。）<br>
>import pymysql<br>
from DBUtils.PooledDB import PooledDB<br>
POOL = PooledDB(<br>
　　　　　creator=pymysql,　　# 使用链接数据库的模块<br>
　　　　　maxconnections=6,　　# 连接池允许的最大连接数，0和None表示不限制连接数<br>
　　　　　mincached=2,　　　　# 初始化时，链接池中至少创建的空闲的链接，0表示不创建<br>
　　　　　maxcached=5,　　　　# 链接池中最多闲置的链接，0和None不限制<br>
　　　　　maxshared=3,　　　　# 链接池中最多共享的链接数量，0和None表示全部共享。PS: 无用，因为pymysql和MySQLdb等模块的 threadsafety都为1，所有值无论设置为多少，_maxcached永远为0，所以永远是所有链接都共享。<br>
　　　　　blocking=True,　　　# 连接池中如果没有可用连接后，是否阻塞等待。True，等待；False，不等待然后报错<br>
　　　　　maxusage=None,　　　# 一个链接最多被重复使用的次数，None表示无限制<br>
　　　　　setsession=[],　　　# 开始会话前执行的命令列表。如：["set datestyle to ...", "set time zone ..."]<br>
　　　　　ping=0,　　　　　　　# ping MySQL服务端，检查是否服务可用，如：0 = None = never, 1 = default = whenever it is requested, 2 = when a cursor is created, 4 = when a query is executed, 7 = always<br>
　　　　　host="127.0.0.1",<br>
　　　　　port=3306,<br>
　　　　　user="root",<br>
　　　　　password="",<br>
　　　　　database="code_record"<br>
　　　　)
def func():<br>
　　\# 检测当前正在运行连接数的是否小于最大链接数，如果不小于则：等待或报raise TooManyConnections异常<br>
　　\# 否则<br>
　　\# 则优先去初始化时创建的链接中获取链接 SteadyDBConnection。<br>
　　\# 然后将SteadyDBConnection对象封装到PooledDedicatedDBConnection中并返回。<br>
　　\# 如果最开始创建的链接没有链接，则去创建一个SteadyDBConnection对象，再封装到PooledDedicatedDBConnection中并返回。<br>
　　\# 一旦关闭链接后，连接就返回到连接池让后续线程继续使用。<br>
　　conn = POOL.connection()<br>
　　cursor = conn.cursor()<br>
　　cursor.execute('select * from userinfo')<br>
　　result = cursor.fetchall()<br>
　　print(result)<br>
　　conn.close()<br>
func()

　　在 uwsgi 中，每个 http 请求都会分发给一个进程，连接池中配置的连接数都是一个进程为单位的（即上面的最大连接数，都是在一个进程中的连接数），而如果业务中，一个 http 请求中需要的 sql 连接数不是很多的话（其实大多数都只需要创建一个连接），配置的连接数配置都不需要太大。<br>

连接池对性能的提升表现在：
>1、在程序创建连接的时候，可以从一个空闲的连接中获取，不需要重新初始化连接，提升获取连接的速度<br>
2、关闭连接的时候，把连接放回连接池，而不是真正的关闭，所以可以减少频繁地打开和关闭连接<br>

为什么要使用数据库连接池呢？<br>
　　如果没有连接池，使用pymysql来连接数据库时，单线程应用完全没有问题；但如果涉及到多线程应用那么就需要加锁，一旦加锁那么连接势必就会排队等待（无法并发），当请求比较多时，性能就会降低了。<br>